
## 4. <a id='4_cell'></a>Create MeasureReport(MR) using CCV, Pandas and the PyFHIR models....

This uses Pandas to easily upload patient data from CSV file and create  a MeasureReport (MR). The MR is then converted back the CSV to demonstrat the roundtrip.

- I'm using the [CDC/NHSN csv](https://www.cdc.gov/nhsn/pdfs/covid19/import-covid19-data-508.pdf) format with sample file `c19_data.csv` containing data for Hospital X period of several days
- convert csv to Panda dataframe
- create MR from each row in dataframe
    -  based upon this [Measure definition](file:///Users/ehaas/Documents/FHIR/IG-template4/docs/Measure-cdc-covid.html)
    -  using static metadata for now
    -  
- Validate MR
- Bundle the MRs Save and Post to a reference server
- Convert bundle back to the CDC/NHSN csv format
- Save CSV file

In [1]:
cd ~/pyFHIR_models # This is to make sure in right directory for Binder

[Errno 2] No such file or directory: '/Users/ehaas/pyFHIR_models # This is to make sure in right directory for Binder'
/Users/ehaas/Documents/Python/MyBinder/pyFHIR_models


### import all the modules you need

In [1]:
from fhir_model_generator.model import measurereport, bundle, fhirdate, fhirreference, group, patient, identifier, contactpoint, address
from json import dumps
from pandas import *
from pathlib import Path
from requests import get, post
from datetime import datetime, date, timedelta
import uuid
from IPython.display import display as Display, HTML, Markdown

### Validate,Save,Bundle,POST Definitions (functions)
-used by main process

In [10]:
#in_file = 'c19_data.csv'  # max file with all fields filled out
in_file = 'c19_data_min.csv'  # min file with only requiired fields filled out

#ref_server ='http://hapi.fhir.org/baseR4'
ref_server ='http://test.fhir.org/r4'
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
params = dict(
          )
out_path = 'MR_bundle'

def isNaN(num):
    return num == num

def validate_me(pyfhir_res):
    print(f'validating to {ref_server}...')
    r = post(f'{ref_server}/{pyfhir_res.resource_type}/$validate',
             params = params, headers = headers, data = dumps(pyfhir_res.as_json())
            )
    display(HTML(
        '<h1>Validation output</h1>'
        f'<h3>Status Code = {r.status_code}</h3>'
        f'{r.json()["text"]["div"]}'
        ))
    
   
def bundle_me(pyfhir_res, fhir_bundle=None):
    file_ts = datetime.utcnow().strftime('%Y%m%d%H%M%S%f')
    new_urn = uuid.uuid1().urn # new urn for resource
    pyfhir_res.id = new_urn[9:]
    e = bundle.BundleEntry()
    e.fullUrl = new_urn
    e.resource = pyfhir_res
    e.request = bundle.BundleEntryRequest()
    e.request.method = 'POST'
    e.request.url = pyfhir_res.resource_type
    if fhir_bundle: #add entry
        pass
    else:  # create transaction bundle
        bundle_type = 'transaction'
        bundle_id = f'cdc-c19-groups-{file_ts}'   
        fhir_bundle = bundle.Bundle(
            dict(
                id = bundle_id,
                type = bundle_type,
                timestamp = pyfhir_res.date.as_json(),
                entry = [],
            )
        )
    fhir_bundle.entry.append(e)
    return(fhir_bundle)
    
def save_me(pyfhir_res):
    print('...........saving to file............')
    #save in out_path
    path = Path.cwd() / out_path / f'{pyfhir_res.resource_type}-{pyfhir_res.id.lower()}.json'
    path.write_text(dumps(pyfhir_res.as_json(), indent=4))
 

def post_me(pyfhir_res):
    print(f'posting to {ref_server} as transaction bundle ...')
    r = post(f'{ref_server}',
          params = params, headers = headers, data = dumps(pyfhir_res.as_json())
            )
    try:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
            '===============================================<br /><br /><br />'
            ))
    except KeyError:
        display(HTML(
            '<h1>Post Response</h1>'
            f'<h3>Status Code = {r.status_code}</h3><br />'
            f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
            #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
            '===============================================<br /><br /><br />'
            ))   



### Get local file and Read into Pandas DataFrame

- convert date string to date object

In [11]:
df = read_csv(in_file)
df['collectiondate'] = to_datetime(df.collectiondate) # convert date to date-time
df['collectiondate'].dt.date
df

,collectiondate,numTotBeds,numbeds,numBedsOcc,numICUBeds,numICUBedsOcc,numVent,numVentUse,numC19HospPats,numC19MechVentPats,numC19HOPats,numC19OverflowPats,numC19OFMechVentPats,numC19Died
0,2020-03-30,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-31,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-01,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-02,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-03,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-04-04,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-04-05,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-04-06,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Initialize the MR Resource

 - start with fixed meta data as dict
 - iterate over the dataframe rows add in period and scores

In [12]:
# ********* Globals ***************

f_now = fhirdate.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
f_now = f_now.as_json()

IP = {
        "coding" : [
          {
            "system" : "http://terminology.hl7.org/CodeSystem/measure-population",
            "code" : "initial-population"
          }
        ],
      }

cdc_c19_groups = {
  "Hospital Bed/ICU Capacity": [
    "numVent",
    "numVentUse"
  ],
  "Hospital Ventilator Capacity": [
    "numTotBeds",
    "numbeds",
    "numBedsOcc",
    "numICUBeds",
    "numICUBedsOcc"
  ],
  "Patient Impact": [
    "numC19HospPats",
    "numC19MechVentPats",
    "numC19HOPats",
    "numC19OverflowPats",
    "numC19OFMechVentPats",
    "numC19Died"
  ]
}

cdc_c19_measure = "http://cdcmeasures.example.org/modules/covid19/20200331/numTotBeds"
NSHN_system = 'urn:oid:2.16.840.1.113883.6.277' # OID for NHSN
NSHN_OID = '2.16.840.1.113883.6.277.123'  # Fake OID


mr_loc_identifier = dict(
    system = NSHN_system,
    value = NSHN_OID
    )

mr_loc = dict(
  identifier = mr_loc_identifier,
  display = "Acme General Hospital",  
    )

mr_meta = dict(
            status = 'complete',
            type = 'individual',
            measure = cdc_c19_measure,
            subject = mr_loc,
            date = f_now,
            reporter = mr_loc,  # for now same as subject
        )

# ********* Globals ***************

my_bundle = None
for i in df.itertuples(index=True):
    print(f'***************** row = {i.Index} *******************')
    mr_period = dict(
        start = str(i.collectiondate.date()),
        end = str(i.collectiondate.date())
    )
    mr_meta.update(dict(period = mr_period))
    mr_groups = []
    for k,v in cdc_c19_groups.items():        
        mr_populations=[]
        for j in v:
            #print(k,j,getattr(i, j)) if getattr(i, j) else print('k,j,"NULL"')
            mr_population =  dict(
                id = j,
                code = IP,
                count = getattr(i, j, None) if isNaN(getattr(i, j, None)) else None
                )
            mr_populations.append(mr_population)             
        mr_group = dict(
            code = {'text':k},
            population = mr_populations,      
            )
        mr_groups.append(mr_group)
    mr_meta.update(dict(group = mr_groups))
    my_mr = measurereport.MeasureReport(mr_meta, strict=False)  # initialize MR instance
    #print(dumps(my_mr.as_json(),indent=4))
    validate_me(my_mr)
    my_bundle = bundle_me(my_mr,my_bundle) # as transaction
print(dumps(my_bundle.as_json(),indent=4))
validate_me(my_bundle)
save_me(my_bundle)
post_me(my_bundle)

***************** row = 0 *******************
validating to http://test.fhir.org/r4...


***************** row = 1 *******************
validating to http://test.fhir.org/r4...


***************** row = 2 *******************
validating to http://test.fhir.org/r4...


***************** row = 3 *******************
validating to http://test.fhir.org/r4...


***************** row = 4 *******************
validating to http://test.fhir.org/r4...


***************** row = 5 *******************
validating to http://test.fhir.org/r4...


***************** row = 6 *******************
validating to http://test.fhir.org/r4...


***************** row = 7 *******************
validating to http://test.fhir.org/r4...


{
    "resourceType": "Bundle",
    "id": "cdc-c19-groups-20200404235001422142",
    "type": "transaction",
    "timestamp": "2020-04-04T23:50:01.064094Z",
    "entry": [
        {
            "fullUrl": "urn:uuid:ffbe4642-76ce-11ea-a7b1-a4d18ccf5172",
            "resource": {
                "resourceType": "MeasureReport",
                "id": "ffbe4642-76ce-11ea-a7b1-a4d18ccf5172",
                "status": "complete",
                "type": "individual",
                "measure": "http://cdcmeasures.example.org/modules/covid19/20200331/numTotBeds",
                "subject": {
                    "identifier": {
                        "system": "urn:oid:2.16.840.1.113883.6.277",
                        "value": "2.16.840.1.113883.6.277.123"
                    },
                    "display": "Acme General Hospital"
                },
                "date": "2020-04-04T23:50:01.064094Z",
                "reporter": {
                    "identifier": {
                      

...........saving to file............
posting to http://test.fhir.org/r4 as transaction bundle ...


### Convert back to CSV

- create new Dataframe based on the old one ( this is cheating )
- For each bundle add row to Dataframe
- save as CSV

In [15]:
new_df = df.iloc[0:0,:].copy()
for i, entry in enumerate(my_bundle.entry):
    new_row = {}
    mr = entry.resource
    new_row['collectiondate']= mr.period.start.as_json()
    for group in mr.group:
        for pop in group.population:
            new_row[pop.id]= pop.count            
    new_df = new_df.append(new_row, ignore_index=True)

new_df['collectiondate'] = to_datetime(new_df.collectiondate) # convert date to date-time
new_df['collectiondate'] = new_df['collectiondate'].dt.strftime('%m/%d/%Y')
new_df.fillna(value=np.nan, inplace=True) # # convert None to NaN
new_df

,collectiondate,numTotBeds,numbeds,numBedsOcc,numICUBeds,numICUBedsOcc,numVent,numVentUse,numC19HospPats,numC19MechVentPats,numC19HOPats,numC19OverflowPats,numC19OFMechVentPats,numC19Died
0,03/30/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03/31/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04/01/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/02/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04/03/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,04/04/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,04/05/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,04/06/2020,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
path = Path.cwd() / out_path / f'{my_bundle.resource_type}-{my_bundle.id.lower()}.csv'
print(f'....saving {my_bundle.resource_type}-{my_bundle.id.lower()}.csv to file ....')
new_df.to_csv(path, index=False)

....saving Bundle-cdc-c19-groups-20200404235001422142.csv to file ....
